<a href="https://colab.research.google.com/github/vijayrowtula/colab_notebooks/blob/main/VGG16_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code from https://www.kaggle.com/code/willzy/vgg16-with-cifar10/script by YUCHENG. Thanks to the author!

In [2]:
from torchvision import datasets as ds
from torch.utils.data import DataLoader
from torchvision import transforms as ts
import torchvision as tv
import torch
import torch.nn as nn
import math
import numpy as np
from torch.autograd import Variable
from torch import optim
from matplotlib import pyplot as plt
import torch.backends.cudnn as cudnn

In [3]:
transform = ts.Compose(
    [
        ts.ToTensor(),
        ts.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

In [4]:
train_set = ds.CIFAR10(root='dataset/',train=True, transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, num_workers=0)
testset = tv.datasets.CIFAR10(root='dataset/',train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=4, shuffle=True, num_workers=0)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting dataset/cifar-10-python.tar.gz to dataset/
Files already downloaded and verified


In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
class VGGNet(nn.Module):
  def __init__(self, num_classes):
    super(VGGNet, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3, 32, 3, padding=1),  # Conv1
        nn.ReLU(True),
        nn.Conv2d(32, 32, 3, padding=1),  # Conv2
        nn.ReLU(True),
        nn.MaxPool2d(2, 2),               # Pool1
        nn.Conv2d(32, 64, 3, padding=1),  # Conv3
        nn.ReLU(True),
        nn.Conv2d(64, 64, 3, padding=1),  # Conv4
        nn.ReLU(True),
        nn.MaxPool2d(2, 2),                # Pool2
        nn.Conv2d(64, 128, 3, padding=1),  # Conv5
        nn.ReLU(True),
        nn.Conv2d(128, 128, 3, padding=1),  # Conv6
        nn.ReLU(True),
        nn.Conv2d(128, 128, 3, padding=1),  # Conv7
        nn.ReLU(True),
        nn.MaxPool2d(2, 2),                 # Pool3
        nn.Conv2d(128, 256, 3, padding=1),  # Conv8
        nn.ReLU(True),
        nn.Conv2d(256, 256, 3, padding=1),  # Conv9
        nn.ReLU(True),
        nn.Conv2d(256, 256, 3, padding=1),  # Conv10
        nn.ReLU(True),
        nn.MaxPool2d(2, 2),                 # Pool4
        nn.Conv2d(256, 256, 3, padding=1),  # Conv11
        nn.ReLU(True),
        nn.Conv2d(256, 256, 3, padding=1),  # Conv12
        nn.ReLU(True),
        nn.Conv2d(256, 256, 3, padding=1),  # Conv13
        nn.ReLU(True),
        # nn.MaxPool2d(2, 2)                # Pool5
    )
    self.classifier = nn.Sequential(
        nn.Linear(2 * 2 * 256, 512),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(512, 512),
        nn.ReLU(True),
        nn.Dropout(),
        nn.Linear(512, num_classes),
    )
    self._initialize_weights()

  def _initialize_weights(self):
    for m in self.modules():
        if isinstance(m, nn.Conv2d):
            n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
            m.weight.data.normal_(0, math.sqrt(2. / n))
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()
            
  def forward(self, x):
      x = self.features(x)
      x = x.view(x.size(0), -1)
      x = self.classifier(x)
      return x

In [7]:
net = VGGNet(10)
net.cuda()
lr = 1e-3
momentum = 0.9
num_epoch = 50

In [8]:
critierion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)
print('Training with learning rate = %f, momentum = %f ' % (lr, momentum))

Training with learning rate = 0.001000, momentum = 0.900000 


In [13]:
loss_p = np.array([])
e = np.linspace(0, num_epoch, num_epoch)
for t in range(num_epoch):
    running_loss = 0
    running_loss_sum_per_epoch = 0
    total_images = 0
    correct_images = 0
    if t == 25:
        optimizer = optim.SGD(net.parameters(), lr=lr/10, momentum=momentum)
    for i, data in enumerate(train_loader, 0):
        images, labels = data
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())

        optimizer.zero_grad()
        outputs = net(images)
        _, predicts = torch.max(outputs.data, 1)
        loss = critierion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_images += labels.size(0)
        correct_images += (predicts == labels).sum().item()
        loss_data = loss.data
        running_loss += loss_data
        
        running_loss_sum_per_epoch = running_loss + running_loss_sum_per_epoch
        if i % 2000 == 1999:
            print('Epoch, batch [%d, %5d] loss: %.6f, Training accuracy: %.5f' %
                  (t + 1, i + 1, running_loss / 2000, 100 * correct_images / total_images))
            running_loss = 0
            total_images = 0
            correct_images = 0
    cpu_run_loss = running_loss_sum_per_epoch.cpu()
    loss_p = np.append(loss_p, cpu_run_loss)

print('Finished training.')

Epoch, batch [1,  2000] loss: 1.762038, Training accuracy: 29.23750
Epoch, batch [1,  4000] loss: 1.685259, Training accuracy: 31.30000
Epoch, batch [1,  6000] loss: 1.651386, Training accuracy: 34.30000
Epoch, batch [1,  8000] loss: 1.597267, Training accuracy: 36.90000
Epoch, batch [1, 10000] loss: 1.556578, Training accuracy: 40.20000
Epoch, batch [1, 12000] loss: 1.495779, Training accuracy: 43.33750
Epoch, batch [2,  2000] loss: 1.394160, Training accuracy: 47.72500
Epoch, batch [2,  4000] loss: 1.355479, Training accuracy: 50.17500
Epoch, batch [2,  6000] loss: 1.265778, Training accuracy: 53.55000
Epoch, batch [2,  8000] loss: 1.255008, Training accuracy: 54.38750
Epoch, batch [2, 10000] loss: 1.184988, Training accuracy: 57.27500
Epoch, batch [2, 12000] loss: 1.162941, Training accuracy: 59.17500
Epoch, batch [3,  2000] loss: 1.083994, Training accuracy: 62.08750
Epoch, batch [3,  4000] loss: 1.063135, Training accuracy: 62.62500
Epoch, batch [3,  6000] loss: 1.027741, Training

KeyboardInterrupt: ignored